In [43]:
import tensorflow as tf
import os, time, datetime, copy, timejob
import numpy as np
import pandas as pd
import talib as ta
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model
from IPython.display import Image
pd.options.display.max_columns = 100
pd.options.display.max_rows = 3000
pd.options.display.width = 5000
pd.options.mode.chained_assignment = None

stock_i = 'HK.800000'


dataset = pd.read_csv('Database/' + stock_i + '/' + stock_i + '_min.csv', index_col=0)
dataset.drop(['pe_ratio', 'turnover_rate', 'volume', 'change_rate'], axis=1, inplace=True)

dataset['mid'] = (dataset['open'] + dataset['close'] + dataset['high'] + dataset['low']) / 4
dataset['next 5 trend'] = None
'''
for i in range(len(dataset) - 5):
    dataset['next 5 trend'][i] = dataset['mid'][i + 1:i + 6].mean()

dataset['target'] = dataset['next 5 trend'] - dataset['mid']

for j in range(len(dataset)):
    if dataset['time_key'][j][11:] == '16:00:00':
        dataset.drop(range(j - 4, j + 1), inplace=True)

for k in range(1, 3):
    dataset['%s min ch' %(k)] = dataset['mid'].shift(k - 1) - dataset['mid'].shift(k)
'''

dataset['buying power'] = dataset['turnover'] - dataset['turnover'].mean()

dataset['change'] = dataset['next 5 trend'] - dataset['mid']

dataset['op-close'] = dataset['open'] - dataset['close']
dataset['hi-low'] = dataset['high'] - dataset['low']

dataset.drop(range(100), inplace=True)
dataset.reset_index(drop=True, inplace=True)
'''
targetData = dataset['target'].copy(deep=True)
targetData = pd.to_numeric(targetData,errors = 'coerce')
dataset.drop(['time_key', 'open', 'close', 'high', 'low', 'mid', 'turnover', 'change', 'next 5 trend', 'target'], axis=1, inplace=True)
'''
print(dataset.head(50))
#print(targetData.head(10))

               time_key      open     close      high       low      turnover         mid next 5 trend  1buying power  2buying power  3buying power  4buying power  5buying power  6buying power  7buying power  8buying power  9buying power  10buying power  11buying power  12buying power  13buying power  14buying power  15buying power  16buying power  17buying power  18buying power  19buying power  20buying power  21buying power  22buying power  23buying power  24buying power  25buying power  26buying power  27buying power  28buying power  29buying power change  op-close  hi-low
0   2020-01-02 11:10:00  28475.00  28477.26  28481.23  28469.89  2.500489e+08  28475.8450         None  -2.065832e+08  -2.065832e+08  -2.065832e+08  -2.065832e+08  -2.065832e+08  -2.065832e+08  -2.065832e+08  -2.065832e+08  -2.065832e+08   -2.065832e+08   -2.065832e+08   -2.065832e+08   -2.065832e+08   -2.065832e+08   -2.065832e+08   -2.065832e+08   -2.065832e+08   -2.065832e+08   -2.065832e+08   -2.065832e+08   -

115952    -3.8060
115953    -4.8900
115954     1.4250
115955    -0.3425
115956    -0.0015
115957     2.3060
115958     8.2810
115959     8.0465
115960    12.8415
115961    19.2940
115962    18.1945
115963    10.3130
115964    -5.9045
115965    -6.0540
115966    -1.1275
115967     9.5505
115968    16.9700
115969    26.6520
115970    27.3405
115971    28.0710
115972    19.0035
115973     4.4000
115974   -16.1895
115975   -14.1350
115976   -13.0360
115977   -13.9620
115978   -12.5990
115979    -6.3985
115980     3.7015
115981     2.6275
115982    -1.2280
115983    -4.1930
115984   -11.5795
115985   -12.9315
115986    -6.1195
115987    -5.7590
115988    -1.2375
115989    -1.1480
115990    -0.2255
115991    -6.2890
115992    -5.5785
115993    -0.2975
115994    -1.7120
115995    -1.0470
115996    -0.6025
115997    -4.7500
115998    -4.0430
115999    -4.7710
116000     0.1405
116001     3.3690
116002     9.8700
116003    10.0660
116004    12.2375
116005    12.6650
116006    12.4185
116007    

In [40]:
data_num = dataset.shape[0]

train_data = dataset.loc[:data_num * 0.8]
test_data = dataset.loc[data_num * 0.8:]
train_target = targetData.loc[:data_num * 0.8]
test_target = targetData.loc[data_num * 0.8:]

#標準化
mean = train_data.mean()
std = train_data.std()
train_data = (train_data - mean) / std
test_data = (test_data - mean) / std

mean_target = train_target.mean()
std_target = train_target.std()
train_target = (train_target - mean_target) / std_target
#test_target = (test_target - mean_target) / std_target

input_train = np.array(train_data)
target_train = np.array(train_target)
input_test = np.array(test_data)
#target_test = np.array(test_target)

print(input_test[:3])
print(target_train[:3])
print(input_test.shape)


114112     -1.7060
114113     -1.3185
114114     -8.6270
114115     -7.2005
114116     -5.6290
114117     -2.2450
114118      2.1555
114119     -0.3460
114120     -3.2195
114121     -3.8245
114122     -9.2500
114123     -9.8505
114124     -9.8420
114125     -6.5840
114126      2.1115
114127     -0.5015
114128      2.0930
114129     10.0610
114130      7.8815
114131      8.5930
114137   -157.7515
114138    -93.3220
114139    -35.0975
114140    -38.5290
114141    -29.6510
114142    -25.8130
114143    -31.4715
114144    -22.7735
114145     -7.9345
114146     -3.8985
114147      8.5595
114148      3.9845
114149    -15.0395
114150     -1.1915
114151      3.5765
114152     -1.4565
114153     17.2765
114154     34.0255
114155     24.8350
114156     13.8480
114157     11.9730
114158     11.0805
114159      3.1115
114160      5.6120
114161     26.6150
114162     17.3270
114163     14.0270
114164     24.4415
114165      9.6290
114166    -15.7015
114167    -23.1445
114168    -24.6805
114169    -2

In [26]:
import tensorflow as tf
from tensorflow import keras

model = keras.models.Sequential(name = 'model-%s' %(stock_i),
                                layers = [keras.layers.Flatten(input_shape = (len(train_data.columns), )),
                                          keras.layers.Dense(64, activation = 'relu'),
                                          keras.layers.Dense(64, activation = 'relu'),
                                          keras.layers.Dense(64, activation = 'relu'),
                                          keras.layers.Dense(1)])

model.summary()

model.compile(optimizer = keras.optimizers.Adam(0.001),
              loss = keras.losses.Huber(delta=1.0),
              metrics = [keras.metrics.MeanAbsoluteError()])

model_dir = 'Tensor/models/'
log_dir = os.path.join('Tensor', 'model-%s' %(stock_i))
model_cbk = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
model_mckp = tf.keras.callbacks.ModelCheckpoint('Tensor/model-%s/Best-model0-%s.h5' %(stock_i, stock_i),
                                             monitor = 'mean_absolute_error', save_best_only = True, mode = 'min')
history = model.fit(input_train, target_train, batch_size = 512, epochs = 300, callbacks = [model_cbk, model_mckp])

Model: "model-HK.800000"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)                2112      
_________________________________________________________________
dense_11 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_12 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 65        
Total params: 10,497
Trainable params: 10,497
Non-trainable params: 0
_________________________________________________________________
Epoch 1/300
220/220 [==============================] - 2s 3ms/step - loss: 0.3053 - mean_absolute_error: 0.6166

In [36]:
import tensorflow as tf
from tensorflow import keras

stock_i = 'HK.800000'
model = keras.models.Sequential(name = 'model-%s' %(stock_i),
                                layers = [keras.layers.Flatten(input_shape = (len(train_data.columns), )),
                                          keras.layers.Dense(64, activation = 'relu'),
                                          keras.layers.Dense(64, activation = 'relu'),
                                          keras.layers.Dense(64, activation = 'relu'),
                                          keras.layers.Dense(1)])

model.load_weights('Tensor/model-HK.800000/Best-model0-HK.800000.h5')
predictions = model.predict(input_test)

testDf = pd.DataFrame(predictions, columns = {'prediction'})
testDf['prediction'] = (testDf['prediction'] * std_target) + mean_target
test_target.reset_index(inplace=True, drop=True)
testDf['next 5'] = test_target

print(testDf.head(500))

     prediction    next 5
0     -3.942208   -1.7060
1      0.627431   -1.3185
2      1.133257   -8.6270
3      0.580290   -7.2005
4     -0.076356   -5.6290
5     -2.922045   -2.2450
6      1.245662    2.1555
7      1.465513   -0.3460
8     -1.261115   -3.2195
9     -2.607444   -3.8245
10     5.019086   -9.2500
11     0.848088   -9.8505
12     2.161147   -9.8420
13    -3.048069   -6.5840
14    -5.905339    2.1115
15     1.268434   -0.5015
16    -1.124646    2.0930
17    -3.160441   10.0610
18     3.192229    7.8815
19     6.418367    8.5930
20    -8.607705 -157.7515
21   -71.621483  -93.3220
22   -26.034632  -35.0975
23   -42.828499  -38.5290
24     9.305214  -29.6510
25   -48.154663  -25.8130
26    35.442848  -31.4715
27   -13.321340  -22.7735
28    15.189219   -7.9345
29    12.910175   -3.8985
30    28.636986    8.5595
31     8.429437    3.9845
32   -31.528904  -15.0395
33   -12.181592   -1.1915
34    34.972378    3.5765
35    30.109211   -1.4565
36    -7.776039   17.2765
37   -16.114

In [ ]:
testDf = pd.DataFrame(index=range(len(predictions)), columns = {'prediction high', 'prediction low'})

for x in range(len(predictions)):
    testDf['prediction high'][x] = predictions[x][0]
    testDf['prediction low'][x] = predictions[x][1]

testDf['prediction high'] = (testDf['prediction high'] * std_target['day_high']) + mean_target['day_high']
testDf['prediction low'] = (testDf['prediction low'] * std_target['day_low']) + mean_target['day_low']

targetData.reset_index(inplace=True, drop = True)
testDf = pd.concat([testDf, targetData], axis=1)

testDf = testDf[['time_key', 'prediction high', 'day_high', 'prediction low', 'day_low']]
print(testDf)

In [ ]:
df = pd.DataFrame(predictions, columns = {'prediction'})


df['prediction'] = (df['prediction'] * std_target['next 5 min']) + mean_target['next 5 min']

df = pd.concat([df, base_data], axis=1)
df = pd.concat([df, test_target], axis=1)

df['next 5 min'] = (df['next 5 min'] * std_target['next 5 min']) + mean_target['next 5 min']

df['pre result-low'] = df['prediction'] - df['low']
df['real result-low'] = df['next 5 min'] - df['low']

df = df[['time_key', 'high', 'low', 'prediction', 'predict result - low', 'next 5 min', 'real result - low']]

print(df.head(1000))


                 time_key      high       low  predict high   predict low  predict high ch  predict low ch  next 1 high ch  next 1 low ch  portfolio
0     2021-06-24 13:18:00  28849.31  28839.54  28847.980469  28832.500000            -1.33           -7.04            1.41           1.36           
1     2021-06-24 13:19:00  28850.72  28840.90  28848.380859  28832.669922            -2.34           -8.23           -0.09          -0.17           
2     2021-06-24 13:20:00  28850.63  28840.73  28849.429688  28834.359375            -1.20           -6.36            0.93          -2.52           
3     2021-06-24 13:21:00  28851.56  28838.21  28848.349609  28833.539062            -3.21           -4.67           -2.78           1.57           
4     2021-06-24 13:22:00  28848.78  28839.78  28847.289062  28832.640625            -1.49           -7.14            5.42           4.31           
5     2021-06-24 13:23:00  28854.20  28844.09  28852.769531  28838.169922            -1.43           -5.92